In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.table("airbnb1.bronze.bronze_data")
display(df)

In [0]:
df.columns

In [0]:
df_silver = df.select(col("id"),col("name"),col("host_id"),col("host_name"),col("host_since"),col("host_location"),col("host_response_rate"),col("host_neighbourhood"),col("host_listings_count"),col("host_identity_verified"),col("neighbourhood_cleansed"),col("zipcode"),col("latitude"),col("longitude"),col("property_type"),col("room_type"),col("accommodates"),col("bathrooms"),col("bedrooms"),col("beds"),col("price"),col("weekly_price"),col("monthly_price"),col("security_deposit"),col("cleaning_fee"),col("guests_included"),col("extra_people"),col("minimum_nights"),col("maximum_nights"),col("number_of_reviews"),col("review_scores_rating"),col("review_scores_accuracy"),col("instant_bookable"),col("cancellation_policy"))

In [0]:
display(df_silver)

In [0]:
df_silver = df_silver.withColumn('host_identity_verified',col("host_identity_verified").cast("boolean"))\
                     .withColumn("price",regexp_replace(col("price"), "[^0-9.]", "").cast("double"))\
                     .withColumn("weekly_price",regexp_replace(col("weekly_price"), "[^0-9.]", "").cast("double"))\
                     .withColumn("monthly_price",regexp_replace(col("monthly_price"), "[^0-9.]", "").cast("double"))\
                     .withColumn("security_deposit",regexp_replace(col("security_deposit"), "[^0-9.]", "").cast("double"))\
                     .withColumn("cleaning_fee",regexp_replace(col("cleaning_fee"), "[^0-9.]", "").cast("double"))\
                     .withColumn("extra_people",regexp_replace(col("extra_people"), "[^0-9.]", "").cast("double"))\
                     .withColumn('instant_bookable',col("instant_bookable").cast("boolean"))

In [0]:
display(df_silver)

In [0]:
df_silver = df_silver.withColumn('Price_category',
    when(col("price") < 100,"Budget")\
    .when((col("price") >= 100) & (col("price") < 250),"Mid-range")
    .when((col("price") >= 250) & (col("price") < 600),"Luxury")
    .otherwise("Premium")
)

In [0]:
for i in df_silver.columns:
    x = df_silver.filter(col(i).isNull()).count()
    print(f"{i}: {x}")

In [0]:
df_silver = df_silver.fillna("Not Available",subset=["host_location","host_neighbourhood","zipcode",])\
                     .fillna(0,subset=["bathrooms","bedrooms","beds","price","weekly_price","monthly_price","security_deposit","cleaning_fee","review_scores_rating","review_scores_accuracy"])

In [0]:
for i in df_silver.columns:
    x = df_silver.filter(col(i).isNull()).count()
    print(f"{i}: {x}")

In [0]:
df_silver = df_silver.withColumnRenamed("id", "listing_id")\
                     .withColumnRenamed("extra_people","cost_extra_people")\
                     .withColumnRenamed("review_scores_rating","rating")\
                     .withColumnRenamed("review_scores_accuracy","rating_accuracy")\
                     .withColumnRenamed("instant_bookable","instant_booking_accept")

In [0]:
display(df_silver)

In [0]:
df_silver.write.format("delta")\
               .mode("overwrite")\
               .saveAsTable("airbnb1.silver.silver_data")